In [7]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from pyecharts.charts import Pie
from pyecharts.charts import Line

from snownlp import SnowNLP  # 情感分析

import re
import jieba
from itertools import chain  # 二维，一维列表转换
from collections import Counter
from wordcloud import WordCloud

In [9]:
file_path = r"/Users/pengyitong/Documents/大数据2102班_彭弋桐_基于小米su7评论的情感分析和数据分析/数据/xiaomi.csv"
df = pd.read_csv(file_path)
df

用户名                                                 内容   点赞量  \
0        SupremeWow                           小米su7强势破圈太火爆了，雷总得抓紧提高产能啊  3025   
1               及米侠                             感觉这次黑稿不太多，看来车圈还是更干净一些！  1725   
2           小麦Elite                                          小米手机1代回忆～  1020   
3              米12U                          雷总辛苦了 感谢您为大家做了一辆Dream car   660   
4     Spider_boy666                                               雷总加电   650   
...             ...                                                ...   ...   
4568          春夏秋冬湖                                              欢迎雷军🍉     0   
4569          四季的暖窝  雷总，能不能发售ARE U OK大字母 运动包 能不能开MI家HOTEL，用SU7接送客人到MI家     0   
4570          米粉玩车局  中国平安的平安车险，未经本人允许，滥用本人姓名、电话、身份证信息，连续一个月车险客服电话骚扰...     0   
4571      浦东机场上空的轰鸣                                     不想体验你们的最新科技，雷嗨     0   
4572      阿罗君君19759                                             歌手2024     1   

                       日期  
0     2024-03-30 12:22:53  
1     2024-03-30 12:23:26  
2     2024-03-30 12:22:38  
3     2024-03-30 12:22:39  
4     2024-03-30 12:23:00  
...                   ...  
4568  2024-05-18 19:58:08  
4569  2024-05-18 19:48:31  
4570  2024-05-18 17:45:00  
4571  2024-05-18 17:42:53  
4572  2024-05-18 17:21:05  

[4573 rows x 4 columns]

In [10]:
df.dtypes

用户名    object
内容     object
点赞量     int64
日期     object
dtype: object

In [11]:
df['点赞量'].describe()

count    4573.000000
mean        5.428165
std        85.513341
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      3025.000000
Name: 点赞量, dtype: float64

In [12]:
null_users = df['用户名'].isnull().sum()
# “用户名”这一列如果有空值就赋零--isnull()，sum()统计一共有多少0
null_users
# np.int64(0)说明没有空值

np.int64(0)

In [13]:
null_likes = df['点赞量'].isnull().sum()
null_likes

np.int64(0)

In [14]:
null_times = df['日期'].isnull().sum()
null_times

np.int64(0)

In [15]:
# 计算每一行中包含空值的数量
null_comments = df['内容'].isnull().sum()
null_comments

np.int64(465)

In [41]:
# 删除 '内容' 列中包含空值的所有行
df = df.dropna()
df

用户名                                                 内容   点赞量  \
0        SupremeWow                           小米su7强势破圈太火爆了，雷总得抓紧提高产能啊  3025   
1               及米侠                             感觉这次黑稿不太多，看来车圈还是更干净一些！  1725   
2           小麦Elite                                          小米手机1代回忆～  1020   
3              米12U                          雷总辛苦了 感谢您为大家做了一辆Dream car   660   
4     Spider_boy666                                               雷总加电   650   
...             ...                                                ...   ...   
4568          春夏秋冬湖                                              欢迎雷军🍉     0   
4569          四季的暖窝  雷总，能不能发售ARE U OK大字母 运动包 能不能开MI家HOTEL，用SU7接送客人到MI家     0   
4570          米粉玩车局  中国平安的平安车险，未经本人允许，滥用本人姓名、电话、身份证信息，连续一个月车险客服电话骚扰...     0   
4571      浦东机场上空的轰鸣                                     不想体验你们的最新科技，雷嗨     0   
4572      阿罗君君19759                                             歌手2024     1   

                       日期  
0     2024-03-30 12:22:53  
1     2024-03-30 12:23:26  
2     2024-03-30 12:22:38  
3     2024-03-30 12:22:39  
4     2024-03-30 12:23:00  
...                   ...  
4568  2024-05-18 19:58:08  
4569  2024-05-18 19:48:31  
4570  2024-05-18 17:45:00  
4571  2024-05-18 17:42:53  
4572  2024-05-18 17:21:05  

[4108 rows x 4 columns]

In [42]:
duplicate_rows = df.duplicated().sum()
# 统计一共有多少重复行
format(duplicate_rows)

'517'

In [43]:
df = df.drop_duplicates()
df

用户名                         内容   点赞量                   日期
0        SupremeWow   小米su7强势破圈太火爆了，雷总得抓紧提高产能啊  3025  2024-03-30 12:22:53
1               及米侠     感觉这次黑稿不太多，看来车圈还是更干净一些！  1725  2024-03-30 12:23:26
2           小麦Elite                  小米手机1代回忆～  1020  2024-03-30 12:22:38
3              米12U  雷总辛苦了 感谢您为大家做了一辆Dream car   660  2024-03-30 12:22:39
4     Spider_boy666                       雷总加电   650  2024-03-30 12:23:00
...             ...                        ...   ...                  ...
4180     我在梦里见过你821                   红米还出汽车吗？     0  2024-03-30 12:22:15
4182        夏天的尾巴_Z               小米fold3为啥造的？     0  2024-03-30 12:22:13
4183         My咪西mi                        suv     0  2024-03-30 12:22:12
4185            缘郎君                   来了来了，雷总。     0  2024-03-30 12:22:01
4187   用户6702293896                         第一     1  2024-03-30 12:21:58

[3591 rows x 4 columns]